    pip install beautifulsoup4
    pip install unidecode

In [1]:
import numpy as np
import unicodedata
import requests
import json
from IPython.core import display
from selenium import webdriver
from bs4 import BeautifulSoup
import os

In [2]:
path = '../data/camptocamp/outiing_list/'
files = os.listdir(path)
files.sort()
files = [f for f in files if not '.npy' in f]
print files

['2006-02-12.txt', '2008-02-14.txt', '2010-03-07.txt', '2012-05-11.txt', '2014-03-29.txt', '2016-03-30.txt', '2018-12-01.txt']


In [170]:
idx_file = -6

In [171]:
file = open(path + files[idx_file])

urls = file.readlines()
urls = {int(url.split('/')[-3]): url for url in urls}

In [172]:
def get_countries(d):
    try:
        return [s['locales'][0]['title'] 
                for s in d['associations']['routes'][0]['areas'] if s['area_type'] == 'country']
    except:
        print 'failed get country for ', d['document_id']
        return []

In [173]:
def is_in_switzerland(d):
    return len(set(get_countries(d)).intersection(
            set(['Schweiz', 'Suitza', 'Su\xefssa', 'Svizzera', 'Switzerland', 'Suisse']))) > 0

In [177]:
countries = []
for d in data_outings.values():
    countries += get_countries(d)

set(countries)

{u'Allemagne',
 u'Austria',
 u'Autriche',
 u'Deutschland',
 u'France',
 u'Francia',
 u'Italia',
 u'Italie',
 u'Liechtenstein',
 u'Schweiz',
 u'Suisse',
 u'Suiza',
 u'Svizzera',
 u'Switzerland'}

In [178]:
# for f in os.listdir('/home/pignate/Downloads/'):
#     if not 'kml' in f and not 'gpx' in f:
#         continue
#     idx = int(f.split('.')[0])
#     if idx in data_outings and not is_in_switzerland(data_outings[idx]):
#         print idx
#         os.remove('/home/pignate/Downloads/' + f)

In [179]:
data_outings = np.load(path + files[idx_file].split('.')[0] + '_data_dict.npy')[()]

In [38]:
driver = webdriver.Chrome()
driver.implicitly_wait(0.1)

In [180]:
for idx, url in urls.iteritems():
    if idx not in data_outings:
        print 'Did not find %s' % url
        continue
    if data_outings[idx]['has_gps'] and is_in_switzerland(data_outings[idx]):
        print idx, ' has gps'
        driver.get(url)
        try:
            buttons = driver.find_elements_by_class_name('green-btn')
            for button in buttons:
                button.click()
        except:
            pass

98777  has gps
98842  has gps
Did not find https://www.camptocamp.org/outings/98888/fr/l-asti-en-boucle-montee-face-nw-descente-face-ne

108032  has gps
109321  has gps
109942  has gps
110408  has gps
110483  has gps
110566  has gps
110649  has gps
111009  has gps
111220  has gps
111270  has gps
111325  has gps
112065  has gps
114201  has gps
114613  has gps
115303  has gps
115527  has gps
116145  has gps
116256  has gps
116812  has gps
116842  has gps
577915  has gps
94806  has gps
95049  has gps
95253  has gps
98211  has gps


In [4]:
driver.get('https://www.camptocamp.org/outings/1061088/fr/tochuhorn-arete-w-voie-normale')

In [5]:
def extract_data_dict_from_html(html, url):
    text = html.get_text()
    
    # if text.find('Download track') != -1:
    #     print "Found gps"
    #     driver.get(url)
    #     try:
    #         buttons = driver.find_elements_by_class_name('green-btn')
    #         for button in buttons:
    #             button.click()
    #     except:
    #         pass
        
    idx_start = text.find('documentData')
    idx_end = text.find('document.querySelector')
    s = (text[idx_start+15:idx_end]).split(');')[0]
    d = json.loads(s)
    d_add = {
        q: filter(lambda x : len(x), 
                  text[text.find(q):text.find(q)+200].split(':')[1].split('\n'))[0].replace(' ','')
        for q in ['snow_quality', 'avalanche_signs', 'quality'] if text.find(q) > 0
    }
    d.update(**d_add)
    d['has_gps'] = text.find('Download track') != -1
    return d

In [6]:
def get_data_dict(url):
    try:
        r = requests.get(url, timeout=5)
        return extract_data_dict_from_html(BeautifulSoup(r.content, 'html.parser'), url)
    except:
        return False

### Loading all data 

### Find url with gps 

In [39]:
url_with_gps = []
url = urls[0]

In [40]:
idx = urls.index(url)

In [41]:
idx

0

In [42]:
for url in urls[idx:]:
    display.clear_output(wait=True)
    try:
        print 'loaded', url
        d = get_data_dict(url)
        if d:
        	data_dict[d['document_id']] = d
        else:
        	print 'Failed to load', url
    except:
        print 'Failed to load', url
    

loaded https://www.camptocamp.org/outings/87356/fr/chasseron-traversee-sn-ns

